In [ ]:
from tweepy.streaming import StreamListener #class alows to listen based certain words
from tweepy import OAuthHandler #auth based on ID in other file
from tweepy import Stream #stream the tweets
from tweepy import API #Twitter API
from tweepy import Cursor 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from textblob import TextBlob
from scipy.stats import gaussian_kde
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

In [ ]:
class AuthenticateTwitter():
    #API access
    def AuthenticTwitterApp(self):
        Authenticate = OAuthHandler("CONSUMER TOKEN","CONSUMER S TOKEN")
        Authenticate.set_access_token("ACCESS TOKEN","ACCESS S TOKEN")
        
        return Authenticate

In [ ]:
class ClientTwitter():
    #API Set up
    def __init__ (self, user_twitter=None):
        self.Authenticate = AuthenticateTwitter().AuthenticTwitterApp()
        self.twitter_client = API(self.Authenticate) 
        self.user_twitter = user_twitter
        
    def Twitter_API(self):
        
        return self.twitter_client

In [ ]:
class AnalyseTweetTwitter():
        
    def D_F_Tweets(self, TTweets):
        #DATA CLEANSING
        #SHOWS TWEET
        dftweet = pd.DataFrame(data=[tweet.text for tweet in TTweets], columns=['TTweets']) 
        #SHOWS SENTIMENT OF TWEET        
        dftweet['TextBlob SA'] = np.array([SentimentTwitter().TextBlob_Tweet_Analyse_Sent(tweet) for tweet in dftweet['TTweets']])
        dftweet['Vader SA'] = np.array([SentimentTwitter().Vader_Tweet_Analyse_sent(tweet) for tweet in dftweet['TTweets']])
        #SHOWS HOW MANY LIKES
        dftweet['Likes']= np.array([tweet.favorite_count for tweet in TTweets])
        #WHEN CREATED
        dftweet['Created']= np.array([tweet.created_at for tweet in TTweets])
        #SHOWS SOURCE
        dftweet['Source of tweet']= np.array([tweet.source for tweet in TTweets])
        #SHOWS HOW MANY RETWEETS
        dftweet['Retweets']= np.array([tweet.retweet_count for tweet in TTweets])
        
        return dftweet 

In [ ]:
class SentimentTwitter():
    
    #Cleansing of the tweet
    def Tweet_Cleansing(self, tweet):
        #takes out the characters that dont need to be there
        return ' '.join(re.sub("(\w+:\/\/\S+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet).split())
    
    def TextBlob_Tweet_Analyse_Sent(self, tweet):
        TextBlobAnalysedTweet = TextBlob(self.Tweet_Cleansing(tweet))#textblob gets the clean tweet and analyse's the tweet
        if TextBlobAnalysedTweet.sentiment.polarity >= 0.5:
            return 1
        elif TextBlobAnalysedTweet.sentiment.polarity > 0 and TextBlobAnalysedTweet.sentiment.polarity < 0.5:
            return 0.5
        elif TextBlobAnalysedTweet.sentiment.polarity == 0:
            return 0
        elif TextBlobAnalysedTweet.sentiment.polarity < 0 and TextBlobAnalysedTweet.sentiment.polarity > 0.5:
            return -0.5
        else:
            return -1
    
    def Vader_Tweet_Analyse_sent(self, tweet):
        VaderAnalysedTweet = SIA()
        vs = VaderAnalysedTweet.polarity_scores(self.Tweet_Cleansing(tweet))
        if vs['compound'] >= 0.5:
            return 1
        elif vs['compound'] > 0 and vs['compound'] < 0.5:
            return 0.5
        elif vs['compound'] == 0:
            return 0
        elif vs['compound'] < 0 and vs['compound'] > 0.5:
            return -0.5
        else:
            return -1
    

In [ ]:
if __name__ == "__main__":
    twitter_client = ClientTwitter()
    analyse_tweet = AnalyseTweetTwitter()
    api = twitter_client.Twitter_API()
    
    TTweets = api.user_timeline(screen_name = "BorisJohnson", count = 20)
    Tweets = api.user_timeline(screen_name = "jeremycorbyn", count = 20)
    
    #TTweets = api.search(q = "cats", lang = "en", result_type = "recent", count = 20)
    
    
    dftweet = analyse_tweet.D_F_Tweets(TTweets) #gets what you want out of the tweet
    dftweet2 = analyse_tweet.D_F_Tweets(Tweets)
    print(dftweet)
    print(dftweet2)

In [ ]:
#Time Series
LikesToTime = pd.Series(data = dftweet['Likes'].values, index = dftweet['Created'])
LikesToTime.plot(figsize=(9,3), color='b')
LikesToTime = pd.Series(data = dftweet2['Likes'].values, index = dftweet2['Created'])
LikesToTime.plot(figsize=(9,3), color='r')
plt.title('Likes in relation to time')
plt.show()

In [ ]:
rng = dftweet['TextBlob SA']
x = dftweet['Created']
y = dftweet['Likes']
colors = rng 
sizes = 100 + (rng * 20)  

rng2 = dftweet2['TextBlob SA']
x2 = dftweet2['Created']
y2 = dftweet2['Likes']
colors2 = rng2
sizes2 = 100 + (rng2 * 20)


plt.scatter(x, y, c=colors, s=sizes, alpha=0.3,
            cmap='viridis')
plt.colorbar();
plt.scatter(x2, y2, c=colors2, s=sizes2, alpha=0.3,
            cmap='Set3')
plt.colorbar();  # show color scale

In [ ]:
SAToTime = pd.Series(data = dftweet['TextBlob SA'].values, index = dftweet['TTweets'])
SAToTime.plot(figsize=(9,3), color='b')
SAToTime = pd.Series(data = dftweet['Vader SA'].values, index = dftweet['TTweets'])
SAToTime.plot(figsize=(9,3), color='r')
plt.title('Difference in SA between the programs')
plt.show()